In [ ]:
import panel as pn
from bokeh.plotting import figure
import holoviews as hv
from holoviews.streams import Params
import param
hv.extension('bokeh')
pn.extension()

In [ ]:
def make_volume():
    """This function makes a 3D array of data, a brain scan"""
    from skimage import io
    vol = io.imread(
#         "https://s3.amazonaws.com/assets.datacamp.com/blog_assets/" # Since the file has been downloaded
        "attention-mri.tif")
    return vol


TOOLTIPS = [
    ("x", "$x"),
    ("y", "$y"),
    ("value", "@image"),
]
shape = (157, 189, 68)
volume = make_volume()
inverted = False


In [ ]:
def make():
    """Here we make a figure widget with a 2D image and put it in the layout"""
    global inverted
    p1 = figure(width=400, height=400, tools='hover,wheel_zoom',
                tooltips=TOOLTIPS,
                x_range=[0, 157], y_range=[0, 189])
    if inverted:
        p1.image(image=[volume[:, ::-1, 30]], x=[0], y=[0], dw=[157], dh=[189])
    else:
        p1.image(image=[volume[:, :, 30]], x=[0], y=[0], dw=[157], dh=[189])
    widgets[1] = p1
    inverted = not inverted

In [ ]:
button = pn.widgets.Button(name='Invert')
button.on_click(lambda *_: make())
widgets = pn.Column(button, button)
make()
widgets


In [ ]:
volume.shape

### Simple Image

In [ ]:
hv.Image(volume[:,:,0])

In [ ]:
def get_brain_image(axes = 'XY',index = 0):
    if axes == 'XY':
        data = volume[:,:,index] 
    elif axes == 'XZ':
        data = volume[:,index,:]
    elif axes == 'YZ': 
        data = volume[index,:,:]
    
    image = hv.Image(data)
    return image

### Dmap for Index

In [ ]:
dmap = hv.DynamicMap(get_brain_image, kdims=['index'])
dmap.redim.range(index=(0,67))

### Dmap for both axes and Index

In [ ]:
axes_list = ['XY','XZ','YZ']
dmap = hv.DynamicMap(get_brain_image, kdims=['axes','index'])
dmap.redim.values(axes = axes_list,index = range(67))

### Brain Explorer

In [ ]:
class BrainExplorer(param.Parameterized):
    
    cmap = param.ObjectSelector(default ='dimgray',objects=hv.plotting.list_cmaps() )
    
    axes_list = ['XY','XZ','YZ']
    axes = param.ObjectSelector(default = 'XY',objects=axes_list)
    
    index = param.Integer(default=10,bounds = (0,max(volume.shape)-1))
    
    @param.depends('axes','index','cmap')
    def get_brain_image(self):
        axes = self.axes
        index = self.index
        cmap = self.cmap
        if axes == 'XY':
            data = volume[:,:,index] 
        elif axes == 'XZ':
            data = volume[:,index,:]
        elif axes == 'YZ':
            data = volume[index,:,:]

        return hv.Image(data).opts(cmap = cmap,
                                   tools = ['hover'],
                                   height = 500,
                                   width = 500,
                                   xlabel = axes[0]+'-Axis', ylabel = axes[1]+'-Axis')

In [ ]:
b = BrainExplorer()
dmap = pn.Row(hv.DynamicMap(b.get_brain_image),b.param)
dmap

### Dashboard

In [ ]:
dashboard = pn.Column(
    pn.Column('# <pre> Brain Explorer',
              pn.Spacer(height=50)),
    pn.Row(
        pn.Row(dmap[0],
               pn.Spacer(width = 50),
               pn.Column(
                   dmap[1][1],
                   pn.Spacer(height = 30),
                   dmap[1][2],
                   pn.Spacer(height = 30),
                   dmap[1][3])
              )
        )
)

In [ ]:
dashboard.servable()

Deploy this dashboard from the CLI using:

```$ panel serve app.ipynb```